# **DATA PREPROCESSING**

**UNZIP DATASET**

**LIBRARIES**

In [1]:
from PIL import Image
import numpy as np
from numpy import random


import os
import pathlib
import random

**DIRECTORIES**

In [ ]:
# directory of dataset
dir_original = "/content/drive/MyDrive/augmented_waste_dataset/content/waste_dataset/augmented_waste_dataset"


# name of new dataset
dir_processed = "/content/drive/MyDrive/augmented_waste_dataset_splited"



**PARAMETERS**

In [ ]:
# size of new images
size = 225, 264

**EXTRACTION OF DATASET INFORMATION**

In [ ]:
data_dir = pathlib.Path(dir_original)

set_samples = ['train', 'validation', 'test']
print("set_samples: ", set_samples, "\n")

CLASS_NAMES = np.array([item.name for item in sorted(data_dir.glob('*'))])												
print("class: ", CLASS_NAMES, "\n")

N_IMAGES = np.array([len(list(data_dir.glob(item.name+'/*.jpg'))) for item in sorted(data_dir.glob('*'))])			# number of images for class
print("number of images for class: ", N_IMAGES, "\n")

N_samples = np.array([(int(np.around(n*60/100)), int(np.around(n*15/100)), int(np.around(n*25/100))) for n in N_IMAGES])	# number of images for set (train,validation,test)
print("split of dataset: \n ", N_samples, "\n")



**PRE-PROCESSING DATASET**

In [ ]:
# Create the new dataset
# Split Dataset	(also resize and rotate)



# create the dataset folder			***********************************
os.makedirs(dir_processed)

for set_tag in set_samples:
	os.makedirs(dir_processed + '/' + set_tag)

	for class_name in CLASS_NAMES:
		os.makedirs(dir_processed + '/' + set_tag + '/' + class_name)



# SPLIT DATASET (and resize)		*************************************
print("Split dataset.....")

i=0
j=0
k=0
for class_name in CLASS_NAMES:												
	
    print("class name: ", class_name)

    contatore_samples = 0
    k=0

    array = sorted(os.listdir(dir_original + '/' + class_name))
    #random.shuffle(array)

    for image_name in array:	                                       
	
        print("image: ", i)
        i=i+1

        if contatore_samples==N_samples[j][k]:									
            k+=1
            contatore_samples=0


        img=Image.open(dir_original +'/'+class_name+'/'+image_name)
        l,_ = img.size
        l=int(l)
        
        
        if l==225 or l==264:
        
            transposed = img.transpose(Image.ROTATE_90)
            transposed.thumbnail(size)
            transposed.save(dir_processed+'/'+set_samples[k]+'/'+class_name+'/'+image_name)
        
        else:
        
            img.thumbnail(size)
            img.save(dir_processed+'/'+set_samples[k]+'/'+class_name+'/'+image_name)

        contatore_samples+=1	

    j+=1




# **MODEL FOR WASTE DATASET**

**LIBRARY**

In [ ]:
import tensorflow as tf
 
from tensorflow import keras
 
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
 
from tensorflow.keras.preprocessing import image_dataset_from_directory
 
import numpy as np
import matplotlib.pyplot as plt
import os
import time

**DIRECTORY**

In [ ]:
# Ditectory for the dataset
 
PATH_DATASET = '/content/drive/MyDrive/augmented_waste_dataset_splited'
 
train_data_dir = os.path.join(PATH_DATASET, 'train')
validation_data_dir = os.path.join(PATH_DATASET, 'validation')
test_data_dir = os.path.join(PATH_DATASET, 'test')
 
 
 
# Names of files to be created
 
PATH_MODELS = '/content/drive/MyDrive/Colab Notebooks/MDPI_Paper'
 
name_model_small = os.path.join(PATH_MODELS, 'model_small,h5')
name_model_large = os.path.join(PATH_MODELS, 'model_large.h5')

**PARAMETERS**

In [ ]:
batch_size = 32
 
nb_train_samples = 207522           # no of training samples
nb_validation_samples = 51880       # no of validation samples
nb_test_samples = 86468             # no of testing samples
 
n_class = 2
 
epochs = 50

# **MODEL (SMALL)**

In [ ]:
start = time.time()
 
# image size (Model Small)
img_width, img_height = 80, 45
 
# input shape
if keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
 
 
 
# ***********************************************************************
# ************        DATASET       *************************************
# ***********************************************************************
 
train_dataset = image_dataset_from_directory(train_data_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_width, img_height),
                                             label_mode='categorical')
 
 
validation_dataset = image_dataset_from_directory(validation_data_dir,
                                                  shuffle=True,
                                                  batch_size=batch_size,
                                                  image_size=(img_width, img_height),
                                                  label_mode='categorical')
 
 
test_dataset = image_dataset_from_directory(test_data_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_width, img_height),
                                            label_mode='categorical')
 
 
# preprocessing: input scaling (./255)
train_dataset = train_dataset.map(lambda images, labels: (images/255, labels))
validation_dataset = validation_dataset.map(lambda images, labels: (images/255, labels))
test_dataset = test_dataset.map(lambda images, labels: (images/255, labels))
 
  
# ***********************************************************************
# **************        MODEL       *************************************
# ***********************************************************************
 
model_small = Sequential()
model_small.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model_small.add(Activation('relu'))
model_small.add(MaxPooling2D(pool_size=(2, 2)))
 
model_small.add(Conv2D(32, (3, 3), padding='same'))
model_small.add(Activation('relu'))
model_small.add(MaxPooling2D(pool_size=(2, 2)))
 
model_small.add(Conv2D(64, (3, 3), padding='same'))
model_small.add(Activation('relu'))
model_small.add(MaxPooling2D(pool_size=(2, 2)))
 
model_small.add(Conv2D(64, (3, 3), padding='same'))
model_small.add(Activation('relu'))
model_small.add(MaxPooling2D(pool_size=(2, 2)))
 
model_small.add(Conv2D(32, (3, 3), padding='same'))
model_small.add(Activation('relu'))
model_small.add(MaxPooling2D(pool_size=(2, 2)))
 
model_small.add(Flatten())
model_small.add(Dense(64))
model_small.add(Activation('relu'))
model_small.add(Dropout(0.5))
model_small.add(Dense(2))           #because we have 2 class
model_small.add(Activation('softmax'))
 
model_small.summary()
 
 
# ***********************************************************************
# *******************        COMPILATION       **************************
# ***********************************************************************
 
 
model_small.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adadelta(learning_rate=1, name='Adadelta'),
            metrics=['accuracy'])
 
 
 
# ***********************************************************************
# *******************        TRAINING       *****************************
# ***********************************************************************
 
 
with tf.device('/device:GPU:0'):
 
  history = model_small.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset)
 
 
 
# ***********************************************************************
# *****************        SAVE MODEL        ****************************
# ***********************************************************************
 
 
model_small.save(name_model_small)
 
 
 
# ***********************************************************************
# ********************        PLOT RESULTS        ***********************
# ***********************************************************************
 
 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
 
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs_range = range(epochs)
 
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy_'+str(img_width)+' x '+str(img_height))
 
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss_'+str(img_width)+' x '+str(img_height))
plt.show()
 
 
 
# ***********************************************************************
# ***********************        TEST        ****************************
# ***********************************************************************
 
with tf.device('/device:GPU:0'):
 
  test_result = model_small.evaluate(test_dataset)
 
  
print("size of images: ", img_width,img_height)
print("test_result: ", test_result)
 
 
print ('Time taken for development model small {} sec\n'.format(time.time() - start))

# **MODEL (LARGE)**

In [ ]:
start = time.time()

# image size (Model Medium)
img_width, img_height = 225, 264

# input shape
if keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)



# ***********************************************************************
# ************        DATASET       *************************************
# ***********************************************************************

train_dataset = image_dataset_from_directory(train_data_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_width, img_height),
                                             label_mode='categorical')


validation_dataset = image_dataset_from_directory(validation_data_dir,
                                                  shuffle=True,
                                                  batch_size=batch_size,
                                                  image_size=(img_width, img_height),
                                                  label_mode='categorical')


test_dataset = image_dataset_from_directory(test_data_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_width, img_height),
                                            label_mode='categorical')


# preprocessing: input scaling (./255)
train_dataset = train_dataset.map(lambda images, labels: (images/255, labels))
validation_dataset = validation_dataset.map(lambda images, labels: (images/255, labels))
test_dataset = test_dataset.map(lambda images, labels: (images/255, labels))


# ***********************************************************************
# **************        MODEL       *************************************
# ***********************************************************************

model_large = Sequential()
model_large.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model_large.add(Activation('relu'))
model_large.add(MaxPooling2D(pool_size=(2, 2)))

model_large.add(Conv2D(32, (3, 3), padding='same'))
model_large.add(Activation('relu'))
model_large.add(MaxPooling2D(pool_size=(2, 2)))

model_large.add(Conv2D(64, (3, 3), padding='same'))
model_large.add(Activation('relu'))
model_large.add(MaxPooling2D(pool_size=(2, 2)))

model_large.add(Conv2D(64, (3, 3), padding='same'))
model_large.add(Activation('relu'))
model_large.add(MaxPooling2D(pool_size=(2, 2)))

model_large.add(Conv2D(32, (3, 3), padding='same'))
model_large.add(Activation('relu'))
model_large.add(MaxPooling2D(pool_size=(2, 2)))

model_large.add(Flatten())
model_large.add(Dense(64))
model_large.add(Activation('relu'))
model_large.add(Dropout(0.5))
model_large.add(Dense(2))			#because we have 2 class
model_large.add(Activation('softmax'))

model_large.summary()


# ***********************************************************************
# *******************        COMPILATION       **************************
# ***********************************************************************


model_large.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adadelta(learning_rate=1, name='Adadelta'),
            metrics=['accuracy'])



# ***********************************************************************
# *******************        TRAINING       *****************************
# ***********************************************************************


with tf.device('/device:GPU:0'):

  history = model_large.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset)



# ***********************************************************************
# *****************        SAVE MODEL        ****************************
# ***********************************************************************


model_large.save(name_model_large)



# ***********************************************************************
# ********************        PLOT RESULTS        ***********************
# ***********************************************************************


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy_'+str(img_width)+' x '+str(img_height))

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss_'+str(img_width)+' x '+str(img_height))
plt.show()



# ***********************************************************************
# ***********************        TEST        ****************************
# ***********************************************************************

with tf.device('/device:GPU:0'):

  test_result = model_large.evaluate(test_dataset)

  
print("size of images: ", img_width,img_height)
print("test_result: ", test_result)


print ('Time taken for development model small {} sec\n'.format(time.time() - start))